## Oceanbolt Python SDK Lesson 6: Port Distances at Scale

For certain maritime analyses, you will require a distance parameter. E.g. to calculate voyage times or estimate bunker consumption. It is often a tedious process to access many port to port distances or calculating the distance from a current position of a vessel to a port. We have made this easy by exposing our proprietary Oceanbolt Port Distance calculator. 

In this lesson, we will demonstrate how. This is Lesson 6 in our Python learning series. You can find past lessons on our blog: https://www.oceanbolt.com/blog/. You can find the full documentation on our distance calculator here: https://python-sdk.oceanbolt.com/distance_v3/distance.html

In [1]:
# Import the relevant libraries

from oceanbolt.sdk.client import APIClient
from oceanbolt.sdk.distance import DistanceCalculator
import pandas as pd

In [2]:
# Create the base API client using your token. Token can be created in the Oceanbolt App (app.oceanbolt.com)
base_client = APIClient("<token>")

### Port to Port Distance
You can use the API to get simple port to port distances. You also have the option to include relevant way points for bunker stops etc.

In [5]:
# Connect to the relevant Oceanbolt data endpoints using the base client object, ie: Distance
distance = DistanceCalculator(base_client).distance(
    locations=[
        {"unlocode": "BRSSZ"},
        {"unlocode": "CNQDG"},
    ]
)
print("There are",round(distance),"nautical miles from Santos to Qingdao")

There are 11211 nautical miles from Santos to Qingdao


### Vessel to Port Distance
Say you wanted instead to find the remaining distance from a particular vessel to a port. 

As an example, you can find the distance from the vessel MAGIC VELA (IMO: 9473327) to Santos.

In [9]:
distance = DistanceCalculator(base_client).distance(
    locations=[
        {"imo": 9473327},
        {"unlocode": "BRSSZ"},
    ]
)
print("There are",round(distance),"nautical miles from MAGIC VELA to Santos")

There are 545 nautical miles from MAGIC VELA to Santos


This is what the query in our dashboard would look like.

<img src="santos_to_9473327.png" width="500" align="center">

### Multiple Ports to Ports
If you have ever had to calculate the distance from multiple ports to multiple ports, you know how tedious this process can be. With our SDK, the process is easy. Simply create a csv file containing all the pairs you want to calculate and process it with the SDK. You can find the steps below. The example shows distances from Santos to 8 different Chinese ports.

In [28]:
# Load CSV into dataframe for easy manipulation
df = pd.read_csv('.../Port_Pairs.csv') #update file path
df

,loadport,disport
0,BRSSZ,CNQDG
1,BRSSZ,CNCFD
2,BRSSZ,CNJTG
3,BRSSZ,CNSGH
4,BRSSZ,CNFCE


In [29]:
# Loop to add distances into dataframe
port_distances = []

for i in range(len(df['loadport'])):
    distance = DistanceCalculator(base_client).distance(
        locations=[
            {"unlocode": df['loadport'][i]},
            {"unlocode": df['disport'][i]},
        ]
    )
    port_distances.append(distance)

port_distances

[11210.651377854478,
 11494.969299168151,
 11481.522713688377,
 10924.825628418632,
 11235.679863256497]

In [30]:
# Appending new column to DataFrame
df2 = pd.DataFrame(port_distances)
df['Distance'] = df2

In [17]:
# Export to CSV
df.to_csv(path_or_buf='<path_to_file>') #update file path